# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [82]:
# Dependencies and Setup
import hvplot.pandas
import holoviews as hv
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('output_data/cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nuuk,64.1835,-51.7216,9.34,76,40,4.63,GL,2024-08-26 19:50:04
1,1,isafjordur,66.0755,-23.1240,11.02,62,0,0.51,IS,2024-08-26 19:50:06
2,2,east london,-33.0153,27.9116,18.86,70,4,2.13,ZA,2024-08-26 19:40:19
3,3,bredasdorp,-34.5322,20.0403,12.35,90,100,5.41,ZA,2024-08-26 19:50:09
4,4,adamstown,-25.0660,-130.1015,18.65,73,100,7.81,PN,2024-08-26 19:50:10


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [131]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'EsriTerrain',
    coastline = True,
    size = 'Humidity',
    color = 'City',
    cmap='bmw',
    height = 800,
    width = 1400
)

# Display the map
city_data_map

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Humidity)
   .Coastline.I :Feature   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria are latitude between -30 and 30, and a humidity level between 30% and 40%
ideal_weather_df = city_data_df.loc[(city_data_df['Lat'] >= -30) & (city_data_df['Lat'] <= 30) & \
                        (city_data_df['Humidity'] <= 50) & (city_data_df['Humidity'] >= 30), :]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,103,tepatitlan de morelos,20.8167,-102.7333,28.09,36,43,3.06,MX,2024-08-26 19:52:36
109,109,aioun,16.6614,-9.6149,36.53,33,90,2.43,MR,2024-08-26 19:52:45
156,156,china,25.7000,-99.2333,35.83,30,5,3.02,MX,2024-08-26 19:53:52
161,161,minas de marcona,-15.2119,-75.1103,17.96,47,41,7.63,PE,2024-08-26 19:53:59
177,177,mancio lima,-7.6142,-72.8958,25.04,50,20,3.60,BR,2024-08-26 19:54:23
214,214,mount isa,-20.7333,139.5000,18.87,42,0,2.57,AU,2024-08-26 19:47:39
218,218,ad dabbah,18.0500,30.9500,35.04,39,77,3.59,SD,2024-08-26 19:55:22
228,228,kasungu,-13.0333,33.4833,17.32,49,0,1.88,MW,2024-08-26 19:55:37
307,307,sultanah,24.4926,39.5857,34.25,38,20,2.57,SA,2024-08-26 19:57:32
350,350,san patricio,28.0170,-97.5169,32.37,45,100,5.87,US,2024-08-26 19:58:34


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({'City': ideal_weather_df['City'], 
                         'Country': ideal_weather_df['Country'], 
                         'Lat': ideal_weather_df['Lat'], 
                         'Lng': ideal_weather_df['Lng'], 
                         'Humidity': ideal_weather_df['Humidity']})

# Add an empty column, 'Hotel Name,' to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
103,tepatitlan de morelos,MX,20.8167,-102.7333,36,None
109,aioun,MR,16.6614,-9.6149,33,None
156,china,MX,25.7000,-99.2333,30,None
161,minas de marcona,PE,-15.2119,-75.1103,47,None
177,mancio lima,BR,-7.6142,-72.8958,50,None
214,mount isa,AU,-20.7333,139.5000,42,None
218,ad dabbah,SD,18.0500,30.9500,39,None
228,kasungu,MW,-13.0333,33.4833,49,None
307,sultanah,SA,24.4926,39.5857,38,None
350,san patricio,US,28.0170,-97.5169,45,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1
params = {
    'categories':categories,
    'limit':limit,
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print('Starting hotel search')

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{lng},{lat},{radius}'
    params['bias'] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = 'https://api.geoapify.com/v2/places'

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as 'No hotel found'.
        hotel_df.loc[index, 'Hotel Name'] = 'No hotel found'

    # Log the search results
    print(f'{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}')

# Display sample data
hotel_df

Starting hotel search
tepatitlan de morelos - nearest hotel: Hotel Magno
aioun - nearest hotel: فندق العيون
china - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
mancio lima - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
ad dabbah - nearest hotel: No hotel found
kasungu - nearest hotel: Kasungu Inn
sultanah - nearest hotel: فندق جلنار
san patricio - nearest hotel: No hotel found
ciudad acuna - nearest hotel: Hotel San Jorge
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
chokwe - nearest hotel: Limpopo Hotel
vilhena - nearest hotel: No hotel found
cuamba - nearest hotel: Pensao Cariaco
puerto rico - nearest hotel: Suizo
dahab - nearest hotel: Lighthouse Hotel
xique xique - nearest hotel: Hotel J. Macedo
guerrero negro - nearest hotel: Plaza sal paraiso
sao paulo de olivenca - nearest hotel: No hotel found
aldama - nearest hotel: Presas del rey


,City,Country,Lat,Lng,Humidity,Hotel Name
103,tepatitlan de morelos,MX,20.8167,-102.7333,36,Hotel Magno
109,aioun,MR,16.6614,-9.6149,33,فندق العيون
156,china,MX,25.7000,-99.2333,30,No hotel found
161,minas de marcona,PE,-15.2119,-75.1103,47,No hotel found
177,mancio lima,BR,-7.6142,-72.8958,50,No hotel found
214,mount isa,AU,-20.7333,139.5000,42,Ibis Styles
218,ad dabbah,SD,18.0500,30.9500,39,No hotel found
228,kasungu,MW,-13.0333,33.4833,49,Kasungu Inn
307,sultanah,SA,24.4926,39.5857,38,فندق جلنار
350,san patricio,US,28.0170,-97.5169,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [182]:
%%capture --no-display
normalized_humidity = (hotel_df['Humidity'] - hotel_df['Humidity'].min()) / (hotel_df['Humidity'].max() - hotel_df['Humidity'].min())
# Configure the map plot
hotel_data_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'CartoEco',
    coastline = True,
    size = 100,
    color = 'City',
    cmap = 'Cool',
    height = 550,
    width = 1200,
    hover_cols = ['Country', 'Hotel Name']
)

# Display the map
hotel_data_map


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (City,Country,Hotel Name)
   .Coastline.I :Feature   [Longitude,Latitude]